In [ ]:
from prophet import Prophet
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

def train_prophet_model(df, input_month, date_column, target_column):
    # Convert the date column to datetime
    df[date_column] = pd.to_datetime(df[date_column])

    # Filter training data to include only the same month across years, excluding the most recent year
    training_data = df[df[date_column].dt.month == input_month]
    training_data = training_data[training_data[date_column].dt.year < training_data[date_column].dt.year.max()]

    # Test data is the most recent year
    test_data = df[df[date_column].dt.year == df[date_column].dt.year.max()]

    # Prepare data for Prophet
    training_data = training_data[[date_column, target_column]].rename(columns={date_column: 'ds', target_column: 'y'})
    test_data = test_data[[date_column, target_column]].rename(columns={date_column: 'ds', target_column: 'y'})

    # Initialize and fit the model
    model = Prophet()
    model.fit(training_data)

    # Make predictions
    forecast = model.predict(test_data[['ds']])

    # Calculate metrics
    mse = mean_squared_error(test_data['y'], forecast['yhat'])
    mae = mean_absolute_error(test_data['y'], forecast['yhat'])

    # Print metrics
    print(f"Mean Squared Error: {mse}")
    print(f"Mean Absolute Error: {mae}")

    # If you want to see the components of the forecast
    # model.plot_components(forecast)

    return forecast

# Example usage
# df = pd.read_csv('your_dataset.csv')
# forecast = train_prophet_model(df, input_month=4, date_column='your_date_column', target_column='your_target_column')


In [ ]:
def make_predictions_with_assumptions(model, assumptions, date_column, future_periods, frequency):
    """
    model: Trained Prophet model
    assumptions: Dictionary containing assumptions for each variable
    date_column: Name of the date column
    future_periods: Number of periods to make predictions for
    frequency: Frequency of the predictions ('D' for daily, 'M' for monthly, etc.)
    """
    # Create a future DataFrame
    future = model.make_future_dataframe(periods=future_periods, freq=frequency)

    # Add the assumptions to the future DataFrame
    for var in assumptions:
        if var in future.columns:
            raise ValueError(f"Variable {var} conflicts with existing columns in the DataFrame.")
        future[var] = assumptions[var]

    # Make predictions
    forecast = model.predict(future)

    # Return the predictions
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Example usage
# assumptions = {'additional_regressor1': [value1, value2, ...], 'additional_regressor2': [value1, value2, ...]}
# predictions = make_predictions_with_assumptions(model, assumptions, 'ds', future_periods=365, frequency='D')
